In [ ]:
import pickle
import pandas
#import pandas.core.indexes.numeric
data = pickle.load(open("springer_dataframe_26_categories.p", "rb"))
print(type(data))
data.head(10)

In [ ]:
df = pandas.read_csv("Classified_Books_Data.csv")
df["classification"].replace("Unclassified", "Physics", inplace=True)
df.head(20)

In [ ]:
df.drop( ["image"], axis='columns', inplace=True)

In [ ]:
df.info()

In [ ]:
df["book_id"] = data['idbook'][:2375] 

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.to_csv("Technical_Books.csv", index=False)

# CREATING TABELS

In [1]:
import pandas as pd
df = pd.read_csv("Technical_Books_Filled.csv")
df.head(5)

,title,author,desc,download_link,pages,publisher,year,language,file,classification,book_id
0,"Soil Mechanics of Earthworks, Foundations and ...",Missing Value,\n This is the third volume...,https://1lib.in/dl/2279969/9cd462,354.0,Elsevier Science,1988.0,english,"PDF, 9.35 MB",Engineering,99500001
1,Foundation Engineering: Design and Constructio...,B.B.K. Huat,\n Residual soils are found...,https://1lib.in/dl/1247565/f9e712,256.0,Taylor & Francis,2006.0,english,"PDF, 86.52 MB",Engineering,99500002
2,Practical Problems in Soil Mechanics and Found...,Sheng Y. Peng,Missing Value,https://1lib.in/dl/2800423/3ccf77,346.0,Elsevier Science Ltd,1985.0,english,"PDF, 9.86 MB",Engineering,99500003
3,Underwater Embankments on Soft Soil A Case His...,William F. van Impe,Missing Value,https://1lib.in/dl/981008/e79e1d,154.0,Missing Value,2007.0,english,"PDF, 3.35 MB",Engineering,99500004
4,Eco- and Ground Bio-Engineering: The Use of Ve...,Alexia Stokes,\n This volume brings toget...,https://1lib.in/dl/901430/e57c96,420.0,Missing Value,2007.0,english,"PDF, 10.67 MB",Engineering,99500005


In [2]:
numerical_columns = df.select_dtypes(include=['number']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

non_numerical_columns = df.select_dtypes(exclude=['number']).columns
df[non_numerical_columns] = df[non_numerical_columns].fillna('Missing Value')


In [3]:
import psycopg2

conn = psycopg2.connect(database = "Technical_Literature", 
                        user = "postgres", 
                        host= '127.0.0.1',
                        password = "25072004",
                        port = 5432)

cur = conn.cursor()

In [4]:
cur.execute("""CREATE TABLE count_of_books(
            label VARCHAR (20) PRIMARY KEY,
            count INTEGER NOT NULL);
            """)

conn.commit()

In [5]:
cur.execute("""CREATE TABLE books(
            label VARCHAR (20),
            book_id INTEGER PRIMARY KEY,
            title VARCHAR (255),
            year INTEGER,
            author VARCHAR (30),
            CONSTRAINT fk_label_count FOREIGN KEY (label) REFERENCES count_of_books(label));
            """)

conn.commit()

In [6]:
cur.execute("""CREATE TABLE aditional_information(
            book_id SERIAL,
            title VARCHAR (260),
            description VARCHAR(600),
            language VARCHAR (30),
            pages INTEGER,
            publisher VARCHAR(100),
            file VARCHAR(30),
            download_link VARCHAR(35),
            CONSTRAINT fk_book_id_books FOREIGN KEY (book_id) REFERENCES books(book_id) ON DELETE CASCADE);
            """)

conn.commit()

# FILLING TABELS

<h3> Filling count_of_books table </h3>

In [7]:
labels = df["classification"].unique()
books = [df[df["classification"] == subject]["classification"].count() for subject in labels]

for label, book in zip(labels, books):
    cur.execute("INSERT INTO count_of_books(label, count) VALUES ('{}', {})".format(label, book))

conn.commit()

In [8]:
cur.execute('SELECT * FROM count_of_books;')
rows = cur.fetchall()
conn.commit()
for row in rows:
    print(row)

('Engineering', 1358)
('Physics', 513)
('Mathematics', 93)
('Science', 62)
('Computer Science', 349)


<h3>Filling books table</h3>

In [9]:
lable = df["classification"]
book_id = df["book_id"]
title = df["title"]
year = df["year"]
author = df["author"]

for i in range(2375):
    cur.execute("""INSERT INTO books(label, book_id, title, year, author) VALUES ('{}', {}, '{}', {}, '{}')""".format(lable[i], book_id[i], title[i].replace("'", ""), year[i], author[i].replace("'", "")[:30]))

conn.commit()

In [10]:
cur.execute('SELECT * FROM books;')
rows = cur.fetchall()
conn.commit()
for row in rows:
    print(row)

('Engineering', 99500001, 'Soil Mechanics of Earthworks, Foundations and Highway Engineering', 1988, 'Missing Value')
('Engineering', 99500002, 'Foundation Engineering: Design and Construction in Tropical Soils (Balkema: Proceedings and Monographs in Engineering, Water and Earth Sciences)', 2006, 'B.B.K. Huat')
('Engineering', 99500003, 'Practical Problems in Soil Mechanics and Foundation Engineering, 2 Wall and Foundation Calculations, Slope Stability', 1985, 'Sheng Y. Peng')
('Engineering', 99500004, 'Underwater Embankments on Soft Soil A Case History: A Case History (Balkema: Proceedings and Monographs in Engineering, Water and Earth Sciences)', 2007, 'William F. van Impe')
('Engineering', 99500005, 'Eco- and Ground Bio-Engineering: The Use of Vegetation to Improve Slope Stability: Proceedings of the First International Conference on Eco-Engineering ... (Developments in Plant and Soil Sciences)', 2007, 'Alexia Stokes')
('Engineering', 99500006, 'Education and Training in Geo-Enginee

<h3> Filling aditional_information table</h3>

In [11]:
book_id = df["book_id"]
title = df["title"]
description = df["desc"]
language = df["language"]
pages = df["pages"]
publisher = df["publisher"]
file = df["file"]
download_link = df["download_link"]

for i in range(2375):
    cur.execute("""INSERT INTO aditional_information(book_id, title, description, language, pages, publisher, file, download_link) VALUES ({}, '{}', '{}', '{}', {}, '{}', '{}', '{}')""".format(book_id[i], 
                                                                                                                                                                 title[i].replace("'", ""), 
                                                                                                                                                                 description[i].replace("'", "")[:255], 
                                                                                                                                                                 language[i],
                                                                                                                                                                 pages[i],
                                                                                                                                                                 publisher[i].replace("'", ""),
                                                                                                                                                                 file[i],
                                                                                                                                                                 download_link[i]))

conn.commit()

In [12]:
cur.execute('SELECT * FROM aditional_information;')
rows = cur.fetchall()
conn.commit()
for row in rows:
    print(row)

(99500001, 'Soil Mechanics of Earthworks, Foundations and Highway Engineering', '\n                    This is the third volume of a handbook which covers the whole field of soil mechanics, discussing deterministic and stochastic theories and methods, and showing how they can be used in conjunction with one another. The first volume di', 'english', 354, 'Elsevier Science', 'PDF, 9.35 MB', 'https://1lib.in/dl/2279969/9cd462')
(99500002, 'Foundation Engineering: Design and Construction in Tropical Soils (Balkema: Proceedings and Monographs in Engineering, Water and Earth Sciences)', '\n                    Residual soils are found in many parts of the world. Like other soils, they are used extensively in construction, being built upon and used as construction materials. Residual soils are formed when the processes of rock weathering pro', 'english', 256, 'Taylor & Francis', 'PDF, 86.52 MB', 'https://1lib.in/dl/1247565/f9e712')
(99500003, 'Practical Problems in Soil Mechanics and Foundatio

In [13]:
cur.close()
conn.close()
